In [48]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import pandas as pd

def descargar_dataset_kaggle(dataset: str, ruta_destino: str) -> str:
    os.makedirs(ruta_destino, exist_ok=True)
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset=dataset, path=ruta_destino, unzip=True)
    print(f"✅ Dataset descargado y extraído en: {ruta_destino}")
    return ruta_destino

# Identificadores en Kaggle
dataset_2425 = "hubertsidorowicz/football-players-stats-2024-2025"
dataset_2526 = "hubertsidorowicz/football-players-stats-2025-2026"

# Descargas en carpetas separadas
ruta_2425 = descargar_dataset_kaggle(dataset_2425, "data/players_2425")
ruta_2526 = descargar_dataset_kaggle(dataset_2526, "data/players_2526")


Dataset URL: https://www.kaggle.com/datasets/hubertsidorowicz/football-players-stats-2024-2025
✅ Dataset descargado y extraído en: data/players_2425
Dataset URL: https://www.kaggle.com/datasets/hubertsidorowicz/football-players-stats-2025-2026
✅ Dataset descargado y extraído en: data/players_2526


In [49]:
def cargar_datos_jugadores(ruta_csv: str, temporada: str) -> pd.DataFrame:
    df = pd.read_csv(ruta_csv, encoding="utf-8-sig")
    df["Season"] = temporada
    return df

# Rutas de los CSV descargados
ruta_csv_2425 = "data/players_2425/players_data-2024_2025.csv"
ruta_csv_2526 = "data/players_2526/players_data-2025_2026.csv"

# Cargar cada dataset con su temporada
df_2425 = cargar_datos_jugadores(ruta_csv_2425, "2024/25")
df_2526 = cargar_datos_jugadores(ruta_csv_2526, "2025/26")

# Concatenar en un único DataFrame
df_jugadores = pd.concat([df_2425, df_2526], ignore_index=True)

# Vista previa
print(df_jugadores.shape)
df_jugadores.head()


(4938, 268)


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Thr,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,Season
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24.0,2000.0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
1,2,Max Aarons,eng ENG,"DF,MF",Valencia,es La Liga,24.0,2000.0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21.0,2003.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,20.0,2004.0,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18.0,2006.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25


In [50]:
# Identificar columnas numéricas (sin imputar todavía)
columnas_numericas = df_jugadores.select_dtypes(include=["number"]).columns

# Vista previa
df_jugadores.head()


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Thr,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,Season
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24.0,2000.0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
1,2,Max Aarons,eng ENG,"DF,MF",Valencia,es La Liga,24.0,2000.0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21.0,2003.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,20.0,2004.0,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18.0,2006.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/25


In [51]:
# Paso 1: Normalizar nombres de ligas y nacionalidades
liga_map = {
    "eng Premier League": "Premier League",
    "es La Liga": "La Liga",
    "de Bundesliga": "Bundesliga",
    "fr Ligue 1": "Ligue 1",
    "it Serie A": "Serie A"
}

# Aplicar reemplazo controlado
df_jugadores["Comp"] = df_jugadores["Comp"].replace(liga_map)

# Mantener solo los últimos 3 caracteres del campo 'Nation'
df_jugadores["Nation"] = df_jugadores["Nation"].astype(str).str[-3:]

# Verificación visual
df_jugadores[["Player", "Nation", "Comp", "Season"]].head(10)


,Player,Nation,Comp,Season
0,Max Aarons,ENG,Premier League,2024/25
1,Max Aarons,ENG,La Liga,2024/25
2,Rodrigo Abajas,ESP,La Liga,2024/25
3,James Abankwah,IRL,Serie A,2024/25
4,Keyliane Abdallah,FRA,Ligue 1,2024/25
5,Yunis Abdelhamid,MAR,Ligue 1,2024/25
6,Himad Abdelli,ALG,Ligue 1,2024/25
7,Mohamed Abdelmoneim,EGY,Ligue 1,2024/25
8,Ali Abdi,TUN,Ligue 1,2024/25
9,Saud Abdulhamid,KSA,Serie A,2024/25


In [52]:
# Paso 2: Clasificación táctica generalizada basada en el primer código de la columna Pos
def clasificar_rol_general(pos):
    if pd.isna(pos):
        return "Otro"
    primera_pos = str(pos).split(",")[0].strip()
    if primera_pos.startswith("GK"):
        return "Portero"
    elif primera_pos.startswith("DF"):
        return "Defensa"
    elif primera_pos.startswith("MF"):
        return "Centrocampista"
    elif primera_pos.startswith("FW"):
        return "Atacante"
    else:
        return "Otro"

# Crear la columna Rol_Tactico directamente desde Pos
df_jugadores["Rol_Tactico"] = df_jugadores["Pos"].apply(clasificar_rol_general)

# Reordenar columna 'Rol_Tactico' justo después de 'Pos'
columnas = df_jugadores.columns.tolist()
columnas.remove("Rol_Tactico")
indice_pos = columnas.index("Pos") + 1
columnas.insert(indice_pos, "Rol_Tactico")

# Aplicar nuevo orden
df_jugadores = df_jugadores[columnas]

# Verificación visual
df_jugadores[["Player", "Pos", "Rol_Tactico", "Comp", "Season"]].head(10)


,Player,Pos,Rol_Tactico,Comp,Season
0,Max Aarons,DF,Defensa,Premier League,2024/25
1,Max Aarons,"DF,MF",Defensa,La Liga,2024/25
2,Rodrigo Abajas,DF,Defensa,La Liga,2024/25
3,James Abankwah,"DF,MF",Defensa,Serie A,2024/25
4,Keyliane Abdallah,FW,Atacante,Ligue 1,2024/25
5,Yunis Abdelhamid,DF,Defensa,Ligue 1,2024/25
6,Himad Abdelli,"MF,FW",Centrocampista,Ligue 1,2024/25
7,Mohamed Abdelmoneim,DF,Defensa,Ligue 1,2024/25
8,Ali Abdi,"DF,MF",Defensa,Ligue 1,2024/25
9,Saud Abdulhamid,DF,Defensa,Serie A,2024/25


In [53]:
import numpy as np
import pandas as pd
import re

# ---------- Helpers ----------
def normalizar_porcentajes_idempotente(df):
    """
    Quita '%' si existe, convierte a numérico y unifica a 0–100.
    Si ya está en 0–100, no lo toca; si está en 0–1, lo multiplica x100.
    Deja NaN los valores negativos o >100.
    """
    df = df.copy()
    for col in df.columns:
        s = df[col]
        # heurística: nombre sugiere % o hay '%' en los valores
        if "%" in col or re.search(r"(pct|rate|ratio|accuracy|perct)", col, re.I) or \
           any(("%" in str(v)) for v in s.dropna().astype(str).head(30).tolist()):
            s2 = s.astype(str).str.replace("%", "", regex=False)
            s2 = pd.to_numeric(s2, errors="coerce")
            if s2.notna().any() and (s2 <= 1.5).mean() > 0.8:
                s2 = s2 * 100.0  # estaba en 0–1
            s2 = s2.mask((s2 < 0) | (s2 > 100))  # fuera de rango
            df[col] = s2
    return df

def es_porcentaje_o_ratio(nombre):
    return ("%" in nombre) or bool(re.search(r"(pct|rate|ratio|accuracy|perct)", nombre, re.I))

# ---------- 1) Copia del DataFrame original ----------
df_per90 = df_jugadores.copy()

# ---------- 1b) Asegurar minutos ----------
if "Min" not in df_per90.columns:
    if "90s" in df_per90.columns:
        df_per90["Min"] = pd.to_numeric(df_per90["90s"], errors="coerce") * 90
    else:
        raise ValueError("No encuentro 'Min' ni '90s' para calcular minutos.")

df_per90["Min"] = pd.to_numeric(df_per90["Min"], errors="coerce")

# ---------- 2) Normalizar % temprano (antes de cálculos) ----------
df_per90 = normalizar_porcentajes_idempotente(df_per90)
print("✅ Porcentajes normalizados (0–100) antes de calcular per-90.")

# ---------- 3) Definir columnas a excluir del cálculo per-90 ----------
columnas_excluir = {
    "Age","Born","Min","90s","MP","Starts","Season","Rol_Tactico"
}

# además, excluimos cualquier columna que ya sea ratio/porcentaje o ya termine en _per90
todas = df_per90.columns.tolist()
excluir_por_naturaleza = {
    c for c in todas
    if es_porcentaje_o_ratio(c) or c.endswith("_per90")
}
columnas_excluir = columnas_excluir.union(excluir_por_naturaleza)

# ---------- 4) Columnas numéricas candidatas a per-90 (solo conteos/tasas acumulativas) ----------
metricas_numericas = [
    c for c in df_per90.select_dtypes(include="number").columns
    if c not in columnas_excluir
]

# ---------- 5) Calcular per-90 solo donde Min>0 (sin inventar 0) ----------
df_valid = df_per90[df_per90["Min"] > 0].copy()
df_per90_per90 = df_valid[metricas_numericas].div(df_valid["Min"], axis=0) * 90
df_per90_per90.columns = [f"{col}_per90" for col in df_per90_per90.columns]

# Reindexar: mantenemos NaN donde no hay minutos o no aplica
df_per90_per90 = df_per90_per90.reindex(df_per90.index)

# ---------- 6) Concatenar sin forzar fillna(0) aquí ----------
df_per90 = pd.concat([df_per90, df_per90_per90], axis=1)

# ---------- 7) Limpieza de infinitos -> NaN (NO los rellenamos con 0 aún) ----------
df_per90.replace([np.inf, -np.inf], np.nan, inplace=True)

# ---------- 8) Vista previa ----------
df_per90.head(10)


✅ Porcentajes normalizados (0–100) antes de calcular per-90.


,Rk,Player,Nation,Pos,Rol_Tactico,Squad,Comp,Age,Born,MP,...,Cmp_stats_keeper_adv_per90,Att_stats_keeper_adv_per90,Att (GK)_per90,Thr_per90,AvgLen_per90,Opp_per90,Stp_per90,#OPA_per90,#OPA/90_per90,AvgDist_per90
0,1,Max Aarons,ENG,DF,Defensa,Bournemouth,Premier League,24.0,2000.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Max Aarons,ENG,"DF,MF",Defensa,Valencia,La Liga,24.0,2000.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Rodrigo Abajas,ESP,DF,Defensa,Valencia,La Liga,21.0,2003.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,James Abankwah,IRL,"DF,MF",Defensa,Udinese,Serie A,20.0,2004.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Keyliane Abdallah,FRA,FW,Atacante,Marseille,Ligue 1,18.0,2006.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Yunis Abdelhamid,MAR,DF,Defensa,Saint-Étienne,Ligue 1,36.0,1987.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Himad Abdelli,ALG,"MF,FW",Centrocampista,Angers,Ligue 1,24.0,1999.0,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Mohamed Abdelmoneim,EGY,DF,Defensa,Nice,Ligue 1,25.0,1999.0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Ali Abdi,TUN,"DF,MF",Defensa,Nice,Ligue 1,30.0,1993.0,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Saud Abdulhamid,KSA,DF,Defensa,Roma,Serie A,25.0,1999.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Estadísticos útiles para interpretación (antes del escalado)
metricas_porteros = [
    "Save%",         # 0–100
    "PSxG+/-_per90",
    "Saves_per90",
    "CS%",           # 0–100
    "Cmp%",          # 0–100
    "Launch%"        # 0–100
]

existentes = [m for m in metricas_porteros if m in df_per90.columns]
desc = df_per90[existentes].describe(percentiles=[.1,.25,.5,.75,.9]).T
desc[["min","10%","25%","50%","75%","90%","max"]]


,min,10%,25%,50%,75%,90%,max
Save%,0.000000,55.600000,63.050000,69.200000,75.725000,81.550000,100.0
PSxG+/-_per90,-2.745763,-0.428214,-0.197017,0.015432,0.183491,0.394867,1.2
Saves_per90,0.000000,1.502542,2.140896,2.750000,3.402632,4.000000,7.5
CS%,0.000000,0.000000,0.000000,23.200000,34.700000,57.300000,100.0
Cmp%,0.000000,63.600000,71.800000,79.400000,85.900000,90.260000,100.0
Launch%,0.000000,18.410000,25.600000,32.400000,41.700000,51.380000,92.3


In [55]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# --- 1) Selección de columnas a escalar ---
# Per-90 (derivadas)
cols_per90 = [c for c in df_per90.columns if c.endswith("_per90")]

# % relevantes (ya normalizados a 0–100 en la celda 5)
percent_candidates = ["Save%", "CS%", "Cmp%", "Launch%"]
cols_percent = [c for c in percent_candidates if c in df_per90.columns]

# Conjunto final de features
features_to_scale = cols_per90 + cols_percent
if not features_to_scale:
    raise ValueError("No hay métricas para escalar. Revisa la creación de *_per90 y columnas %.")

# --- 2) Base y limpieza mínima (mantén NaN para imputar) ---
df_scaled = df_per90.copy()
df_scaled[features_to_scale] = (
    df_scaled[features_to_scale]
        .apply(pd.to_numeric, errors="coerce")
        .replace([np.inf, -np.inf], np.nan)
)

# --- 3) Imputación por grupo (mediana) ---
group_keys = [k for k in ["Comp","Rol_Tactico"] if k in df_scaled.columns]

def _impute_median(df, cols, by=None):
    if by:
        for c in cols:
            if df[c].isna().any():
                df[c] = df.groupby(by)[c].transform(lambda x: x.fillna(x.median()))
    else:
        for c in cols:
            if df[c].isna().any():
                df[c] = df[c].fillna(df[c].median())

_impute_median(df_scaled, features_to_scale, by=group_keys if group_keys else None)
print(f"✅ Imputación aplicada por {group_keys if group_keys else 'mediana global'}")

# --- 4) Escalado MinMax por grupo (PRO) ---
if group_keys:
    parts = []
    for keys, g in df_scaled.groupby(group_keys):
        sc = MinMaxScaler()
        g = g.copy()
        g[features_to_scale] = sc.fit_transform(g[features_to_scale])
        parts.append(g)
    df_scaled = pd.concat(parts, axis=0)
else:
    sc = MinMaxScaler()
    df_scaled[features_to_scale] = sc.fit_transform(df_scaled[features_to_scale])

# --- 5) Chequeo rápido ---
print(df_scaled[features_to_scale].describe().T[["min","max"]])


C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarnin

✅ Imputación aplicada por ['Comp', 'Rol_Tactico']


C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\Betan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered


               min  max
Rk_per90       0.0  1.0
Gls_per90      0.0  1.0
Ast_per90      0.0  1.0
G+A_per90      0.0  1.0
G-PK_per90     0.0  1.0
...            ...  ...
AvgDist_per90  0.0  1.0
Save%          0.0  1.0
CS%            0.0  1.0
Cmp%           0.0  1.0
Launch%        0.0  1.0

[208 rows x 2 columns]


In [56]:
# 1. Lista de columnas deseadas (actualizada y optimizada)
columnas_finales_csv = [
    # Identidad
    "Player", "Nation", "Pos", "Rol_Tactico", "Squad", "Comp", "Season",
    "Age", "Born", "MP", "Starts", "Min", "90s",

    # Producción ofensiva y finalización
    "Gls_per90", "xG_per90", "NPxG_per90",  # NPxG si está disponible
    "Sh_per90", "SoT_per90", "G/SoT_per90",

    # Creatividad e interiores
    "xA_per90", "xAG_per90", "KP_per90", "GCA90_per90", "SCA_per90",
    "1/3_per90", "PPA_per90",

    # Progresión y pase
    "PrgP_per90", "PrgC_per90", "Carries_per90",
    "Cmp%", "Cmp_per90", "TotDist_per90",

    # Defensa y recuperación
    "Tkl+Int_per90", "Int_per90", "Recov_per90", "Blocks_per90", "Clr_per90",

    # Posesión y presión
    "Touches_per90", "Dis_per90", "Pressures_per90", "Err_per90",

    # Portero
    "Save%", "PSxG+/-_per90", "PSxG_per90", "CS%", "Saves_per90", "Launch%",

]

# 2. Filtrar solo columnas que realmente existen en df_scaled
columnas_existentes = [col for col in columnas_finales_csv if col in df_scaled.columns]

# 3. Crear nuevo DataFrame con las columnas finales
df_final = df_scaled[columnas_existentes].copy()

# 4. Vista previa
df_final.head(10)


,Player,Nation,Pos,Rol_Tactico,Squad,Comp,Season,Age,Born,MP,...,Clr_per90,Touches_per90,Dis_per90,Err_per90,Save%,PSxG+/-_per90,PSxG_per90,CS%,Saves_per90,Launch%
26,Junior Adamu,AUT,FW,Atacante,Freiburg,Bundesliga,2024/25,23.0,2001.0,25,...,0.181230,0.098752,0.139806,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
29,Karim Adeyemi,GER,"FW,MF",Atacante,Dortmund,Bundesliga,2024/25,22.0,2002.0,25,...,0.048849,0.126807,0.194696,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
37,Oladapo Afolayan,ENG,"FW,MF",Atacante,St. Pauli,Bundesliga,2024/25,26.0,1998.0,32,...,0.170836,0.129173,0.247102,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
51,Erik Ahlstrand,SWE,"FW,MF",Atacante,St. Pauli,Bundesliga,2024/25,22.0,2001.0,5,...,0.325581,0.146179,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75,Andreas Albers,DEN,FW,Atacante,St. Pauli,Bundesliga,2024/25,34.0,1990.0,14,...,0.225806,0.202765,0.870968,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
119,Mohamed Amoura,ALG,FW,Atacante,Wolfsburg,Bundesliga,2024/25,24.0,2000.0,31,...,0.136198,0.128091,0.065667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
158,Jann-Fiete Arp,GER,"FW,MF",Atacante,Holstein Kiel,Bundesliga,2024/25,24.0,2000.0,23,...,0.157083,0.121419,0.201964,0.081346,NaN,NaN,NaN,NaN,NaN,NaN
211,Ridle Baku,GER,"FW,MF",Atacante,Wolfsburg,Bundesliga,2024/25,26.0,1998.0,15,...,0.179487,0.157753,0.076923,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
227,Samuel Bamba,GER,"FW,MF",Atacante,Bochum,Bundesliga,2024/25,20.0,2004.0,5,...,0.000000,0.328042,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
232,Scott Banks,SCO,"FW,MF",Atacante,St. Pauli,Bundesliga,2024/25,22.0,2001.0,12,...,0.241379,0.118227,0.155172,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Comprobación robusta para columnas numéricas con manejo explícito
metricas_con_problemas = []

# Asegurar que solo analizamos columnas numéricas
columnas_numericas = df_final.select_dtypes(include="number").columns

print(f"🔎 Revisando {len(columnas_numericas)} métricas numéricas...")

for col in columnas_numericas:
    serie = df_final[col]

    if isinstance(serie, pd.Series):
        if pd.isna(serie).sum() == len(serie):
            metricas_con_problemas.append((col, "TODOS son NaN"))
        elif (serie == 0).sum() == len(serie):
            metricas_con_problemas.append((col, "TODOS son 0"))
        elif pd.isna(serie).sum() > 0:
            porcentaje = pd.isna(serie).mean() * 100
            metricas_con_problemas.append((col, f"{porcentaje:.2f}% NaN"))

# Mostrar métricas sospechosas
if metricas_con_problemas:
    print("⚠️ Revisión de métricas con posibles problemas:")
    for metrica, estado in metricas_con_problemas:
        print(f" - {metrica}: {estado}")
else:
    print("✅ Todas las métricas numéricas en df_final tienen datos válidos.")


🔎 Revisando 38 métricas numéricas...
⚠️ Revisión de métricas con posibles problemas:
 - Age: 0.22% NaN
 - Born: 0.22% NaN
 - Min: 0.02% NaN
 - G/SoT_per90: 4.05% NaN
 - Save%: 93.28% NaN
 - PSxG+/-_per90: 93.28% NaN
 - PSxG_per90: 93.28% NaN
 - CS%: 93.28% NaN
 - Saves_per90: 93.28% NaN
 - Launch%: 93.28% NaN


In [58]:
# --- Normalizar NaN de Age, Born y Min (previo a filtro/export) ---

# Age y Born: si faltan -> "N/A" (esto cambia el tipo a texto en esas columnas)
for col in ["Age", "Born"]:
    if col in df_final.columns:
        df_final[col] = df_final[col].astype(object)  # asegura tipo texto
        df_final[col] = df_final[col].where(df_final[col].notna(), "N/A")

# Min: si falta -> 0 (manteniendo tipo numérico)
if "Min" in df_final.columns:
    df_final["Min"] = pd.to_numeric(df_final["Min"], errors="coerce").fillna(0)


In [59]:
# --- FILTRO POR MINUTOS (ROBUSTO) Y EXPORTACIÓN A CSV ---

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

def detectar_o_crear_minutos(df):
    """
    Devuelve (df_con_minutos, nombre_columna_minutos).
    Si no hay 'minutos' pero existe '90s', crea una columna 'Min' = 90 * 90s.
    """
    df = df.copy()
    candidatos_minutos = ["Min", "Minutes", "mins", "minutes", "Minutos"]
    col_min = next((c for c in candidatos_minutos if c in df.columns), None)

    if col_min is None:
        # ¿Tenemos '90s'? (muy común en datos de fútbol)
        if "90s" in df.columns:
            df["Min"] = pd.to_numeric(df["90s"], errors="coerce") * 90
            col_min = "Min"
        else:
            raise ValueError(
                "No encontré una columna de minutos. Busca/renombra una de estas: "
                f"{candidatos_minutos + ['90s']}"
            )

    # Asegurar numérico
    df[col_min] = pd.to_numeric(df[col_min], errors="coerce")
    return df, col_min

def filtrar_por_minutos(df, minimo_minutos=900):
    df, minutos_col = detectar_o_crear_minutos(df)
    df_filtrado = df[df[minutos_col] >= minimo_minutos].copy()
    return df_filtrado, minutos_col

# 👉 Usa el DataFrame correcto (el tuyo se llama df_final)
df_filtrado, minutos_col = filtrar_por_minutos(df_final, minimo_minutos=900)

# Info útil
print(f"Columna de minutos usada: {minutos_col}")
print(f"Filas antes: {len(df_final):,} | Filas después (≥900 min): {len(df_filtrado):,}")

# Exportación a CSV en tu ruta de Windows
destino = Path(r"C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web")
destino.mkdir(parents=True, exist_ok=True)

nombre_archivo = f"scouting_laliga_df_final_{datetime.now().strftime('%Y%m%d')}.csv"
ruta_csv = destino / nombre_archivo

df_filtrado.to_csv(ruta_csv, index=False, encoding="utf-8-sig")
print(f"✅ CSV exportado en: {ruta_csv}")


Columna de minutos usada: Min
Filas antes: 4,938 | Filas después (≥900 min): 1,570
✅ CSV exportado en: C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web\scouting_laliga_df_final_20250923.csv


In [61]:
%pip install pyarrow


   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB 393.8 kB/s eta 0:01:07
   ---------------------------------------- 0.1/26.2 MB 585.1 kB/s eta 0:00:45
   ---------------------------------------- 0.3/26.2 MB 1.4 MB/s eta 0:00:19
   ---------------------------------------- 0.3/26.2 MB 1.4 MB/s eta 0:00:19
    --------------------------------------- 0.4/26.2 MB 1.5 MB/s eta 0:00:18
    --------------------------------------- 0.4/26.2 MB 1.4 MB/s eta 0:00:19
    --------------------------------------- 0.6/26.2 MB 1.4 MB/s eta 0:00:18
    --------------------------------------- 0.6/26.2 MB 1.4 MB/s eta 0:00:18
    --------------------------------------- 0.6/26.2 MB 1.4 MB/s eta 0:00:18
    --------------------------------------- 0.6/26.2 MB 1.3 MB/s eta 0:00:20
    --------------------------------------- 0.6/26.2 MB 1.3 MB/s eta 0:00:20
    -----


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Betan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [67]:
%pip install fastparquet


   ---------------------------------------- 0.0/671.0 kB ? eta -:--:--
    --------------------------------------- 10.2/671.0 kB ? eta -:--:--
    --------------------------------------- 10.2/671.0 kB ? eta -:--:--
   ---- ---------------------------------- 71.7/671.0 kB 558.5 kB/s eta 0:00:02
   ---- ---------------------------------- 71.7/671.0 kB 558.5 kB/s eta 0:00:02
   --------- ---------------------------- 174.1/671.0 kB 871.5 kB/s eta 0:00:01
   ----------------- ---------------------- 286.7/671.0 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 348.2/671.0 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 481.3/671.0 kB 1.5 MB/s eta 0:00:01
   ------------------------------------ --- 614.4/671.0 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 645.1/671.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 671.0/671.0 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Betan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [68]:
# --- Sanitizar tipos antes de guardar a Parquet (evita ArrowKeyError) ---
import pandas as pd

df_out = df_final.copy()

for c in df_out.columns:
    dt = df_out[c].dtype
    dt_str = str(dt)

    # 1) Strings y objetos Arrow -> convertir a 'object' estándar
    #    (evita 'string[pyarrow]' o 'ArrowDtype')
    if "Arrow" in dt_str or dt_str.startswith("string"):
        df_out[c] = df_out[c].astype("object")
        continue

    # 2) Enteros "nullable" (Int64, Int32 de pandas) -> si hay NaN, pasa a float64
    #    (Parquet no admite NaN en enteros puros sin máscara)
    if dt_str.startswith("Int") and df_out[c].isna().any():
        df_out[c] = df_out[c].astype("float64")
        continue

    # 3) Booleanos "nullable" -> pasa a object para preservar NaN
    if dt_str == "boolean":
        df_out[c] = df_out[c].astype("object")
        continue

# Ahora guarda usando pyarrow; si algo falla, cae a CSV de respaldo
from datetime import datetime
from pathlib import Path

fecha = datetime.now().strftime("%Y%m%d")
destino = Path(r"C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web")
destino.mkdir(parents=True, exist_ok=True)
parquet_path = destino / f"scouting_laliga_df_final_{fecha}.parquet"
csv_fallback = destino / f"scouting_laliga_df_final_{fecha}_fallback.csv"

try:
    import fastparquet  # asegura que está disponible
    df_out.to_parquet(parquet_path, index=False, engine="fastparquet")
    print(f"✅ Parquet exportado con fastparquet: {parquet_path}")
    parquet_name = parquet_path.name
except Exception as e:
    print(f"ℹ️ Falló Parquet incluso con fastparquet ({e}). Guardo CSV de respaldo.")
    df_out.to_csv(csv_fallback, index=False, encoding="utf-8-sig")
    print(f"✅ CSV de respaldo exportado: {csv_fallback}")
    parquet_name = csv_fallback.name

ℹ️ Falló Parquet incluso con fastparquet (Error converting column "Age" to bytes using encoding None. Original error: could not convert string to float: 'N/A'). Guardo CSV de respaldo.
✅ CSV de respaldo exportado: C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web\scouting_laliga_df_final_20250923_fallback.csv


In [70]:
# ==== GUARDAR DATASET (PARQUET con fastparquet + CSV fallback) ====
import pandas as pd
import numpy as np
import json
from datetime import datetime
from pathlib import Path

# 0) Copia de trabajo
df_out = df_final.copy()

# 1) Ajustes específicos para Parquet
#    - Age: numérico con NaN (NO "N/A")
if "Age" in df_out.columns:
    df_out["Age"] = pd.to_numeric(df_out["Age"].replace("N/A", np.nan), errors="coerce")

#    - Born: forzamos texto (si venía NaN, lo dejamos como "N/A" para lectura humana)
if "Born" in df_out.columns:
    df_out["Born"] = df_out["Born"].astype(str)
    df_out["Born"] = df_out["Born"].replace({"nan": "N/A", "NaN": "N/A"})

#    - Season u otras columnas Period/Categorical -> texto
for c in df_out.columns:
    s = df_out[c]
    if pd.api.types.is_period_dtype(s):
        df_out[c] = s.astype(str)
    elif pd.api.types.is_categorical_dtype(s):
        df_out[c] = s.astype(str)

# 2) Rutas
destino = Path(r"C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web")
destino.mkdir(parents=True, exist_ok=True)
fecha = datetime.now().strftime("%Y%m%d")
parquet_path = destino / f"scouting_laliga_df_final_{fecha}.parquet"
csv_fallback = destino / f"scouting_laliga_df_final_{fecha}_fallback.csv"

# 3) Guardado principal con fastparquet
try:
    import fastparquet  # asegúrate de haber hecho %pip install fastparquet
    df_out.to_parquet(parquet_path, index=False, engine="fastparquet")
    print(f"✅ Parquet exportado con fastparquet: {parquet_path}")
    main_name = parquet_path.name
except Exception as e:
    print(f"ℹ️ Falló Parquet incluso con fastparquet ({e}). Guardo CSV de respaldo.")
    df_out.to_csv(csv_fallback, index=False, encoding="utf-8-sig")
    print(f"✅ CSV de respaldo exportado: {csv_fallback}")
    main_name = csv_fallback.name

# 4) Diccionario de datos
def construir_diccionario(df):
    filas = []
    for c in df.columns:
        nota = []
        if c.endswith("_per90"): nota.append("por 90’")
        if "%" in c:            nota.append("porcentaje (0–100)")
        if c in ["Min","90s"]:  nota.append("tiempo de juego")
        filas.append({"column": c, "dtype": str(df[c].dtype), "notes": ", ".join(nota)})
    return pd.DataFrame(filas)

data_dict = construir_diccionario(df_out)
dict_path = destino / f"data_dictionary_{fecha}.csv"
data_dict.to_csv(dict_path, index=False, encoding="utf-8-sig")

# 5) Metadatos
metadata = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "rows": int(len(df_out)),
    "cols": int(len(df_out.columns)),
    "files": {
        "main": main_name,
        "data_dictionary": dict_path.name
    },
    "filters": {"minutes_min": 900},
    "scaling": "MinMax por Comp + Rol_Tactico",
    "percent_scale": "0–100",
}
with open(destino / f"metadata_{fecha}.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("✅ Artefactos congelados:")
print(" - Archivo principal:", main_name)
print(" - Diccionario:", dict_path)
print(" - Metadatos:", destino / f"metadata_{fecha}.json")


✅ Parquet exportado con fastparquet: C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web\scouting_laliga_df_final_20250923.parquet
✅ Artefactos congelados:
 - Archivo principal: scouting_laliga_df_final_20250923.parquet
 - Diccionario: C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web\data_dictionary_20250923.csv
 - Metadatos: C:\Users\Betan\OneDrive\Escritorio\Estudios\Masters\Proyectos\Proyecto Web\metadata_20250923.json


C:\Users\Betan\AppData\Local\Temp\ipykernel_36616\2694146252.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_out["Age"] = pd.to_numeric(df_out["Age"].replace("N/A", np.nan), errors="coerce")
C:\Users\Betan\AppData\Local\Temp\ipykernel_36616\2694146252.py:24: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(s):
C:\Users\Betan\AppData\Local\Temp\ipykernel_36616\2694146252.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(s):
